In [3]:
import pandas as pd
import random

# 假設這些 DataFrame 已經被正確地讀入
# label_test_source, label_train_source, sessionid_10_artistid, label_train_target, same_songs_16_20, All_Artists_Top5
# Load your data
label_test_source = pd.read_parquet('label_test_source.parquet')
label_train_source = pd.read_parquet('label_train_source.parquet')
label_train_target = pd.read_parquet('label_train_target.parquet')
same_songs_16_20 = pd.read_csv('song_la3.csv')
sessionid_10_artistid = pd.read_csv('sessionid_10_artistid.csv')
All_Artists_Top5 = pd.read_csv('All_Artists_Top5.csv')

def predict_next_songs(test_source, train_source, train_target, same_songs, artist_10_session, artists_top5):
    predictions = []

    for index, test_session in test_source.groupby('session_id'):
        # 檢查是否所有歌曲相同
        if test_session['song_id'].tolist() == train_source[train_source['session_id'] == index]['song_id'].tolist():
            predicted_songs = train_target[train_target['session_id'] == index].sort_values(by='listening_order')['song_id'].tolist()
        
        # 檢查是否後五首歌相同
        elif index in same_songs['test_session_id'].values:
            train_session_id = same_songs[same_songs['test_session_id'] == index]['train_session_id'].iloc[0]
            predicted_songs = train_target[train_target['session_id'] == train_session_id].sort_values(by='listening_order')['song_id'].tolist()

        # 檢查是否聽了同一藝術家的10首歌
        elif index in artist_10_session['session_id'].values:
            artists = artist_10_session[artist_10_session['session_id'] == index]['listened_artists'].tolist()
            predicted_songs = artists_top5[artists_top5['artist_id'].isin(artists)].nlargest(5, 'play_count')['song_id'].tolist()

        else:
            # 隨機選擇5首歌
            predicted_songs = random.sample(test_session['song_id'].tolist(), 5)

        # 確保預測沒有空值
        if len(predicted_songs) < 5:
            predicted_songs += random.sample(test_session['song_id'].tolist(), 5 - len(predicted_songs))

        predictions.append([index] + predicted_songs[:5])

    return pd.DataFrame(predictions, columns=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'])

# 調用函數並輸出預測結果
predicted_df = predict_next_songs(label_test_source, label_train_source, label_train_target, same_songs_16_20, sessionid_10_artistid, All_Artists_Top5)
predicted_df.to_csv('song_predictionsla2.csv', index=False)